In [ ]:
import tushare as ts
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import talib
from keras.callbacks import TensorBoard
from sklearn.preprocessing import MinMaxScaler
ts.set_token('457a66c9299e50e9e0b2bf6c2f122bb24d560735f01b443158d1c85e')
# 日线接口
pro = ts.pro_api()
# df = pro.daily(ts_code='000001.SH', start_date='20100701', end_date='20190923')
df = ts.pro_bar(ts_code='000300.SH', asset='I', start_date='20100701', end_date='20191201')


In [ ]:
df['trade_date'] = pd.to_datetime(df['trade_date'])
df.set_index('trade_date', inplace=True)

In [ ]:
df=df.sort_index(ascending=True)
plt.figure(figsize = (18,9))
plt.plot(range(df.shape[0]),(df['low']), color='r')
plt.plot(range(df.shape[0]),(df['high']), color = 'b')
plt.xticks(range(0,df.shape[0],60),df.index[::60],rotation=60)
plt.xlabel('trade_date',fontsize=18)
plt.ylabel('Price',fontsize=18)
plt.show()

In [ ]:
high_prices = df.loc[:,'high'].values
low_prices = df.loc[:,'low'].values
mid_prices = (high_prices+low_prices)/2.0
mid_prices.shape

In [ ]:
train_test_split = int(mid_prices.shape[0]*0.95)
train_data = mid_prices[:train_test_split]
test_data = mid_prices[train_test_split:]
train_data = train_data.reshape(-1,1)         #scaler.fit_transform
test_data = test_data.reshape(-1,1)           #scaler.fit_transform

print('%d training and %d total testing instances'%(len(train_data),    
      len(test_data)))

In [ ]:
#Subplot with training data
plt.subplot(1,2,1)
plt.plot(range(train_data.shape[0]),train_data,color='r',label='Training split')
plt.title('Train Data')
plt.xlabel('time')
plt.ylabel('Price')
plt.legend()

#Subplot with test data
plt.subplot(1,2,2)
plt.plot(range(test_data.shape[0]),test_data,color='b',label='Test Split')
plt.title('Test Data')
plt.xlabel('time')
plt.ylabel('Price')
plt.legend()

#adjust layout and plot all
plt.tight_layout()
plt.show()


In [ ]:
#Window size to normalize data in chunks 
normalization_window = 250

#Feature range for normalization
scaler = MinMaxScaler(feature_range=(0, 1))

# Loop over the training data in windows of 250 instances at a time
for i in range(0,len(train_data),normalization_window):
    
    # Fit the scaler object on the data in the current window
    scaler.fit(train_data[i:i+normalization_window,:])
    
    # Transform the data in the current window into values between the chosen feature range (0 and 1)
    train_data[i:i+normalization_window,:] = scaler.transform(train_data[i:i+normalization_window,:])

# normalize the the test data
test_data=scaler.fit_transform(test_data)

In [ ]:
Smoothing = 0.0     #Initialize smoothing value as zero
gamma = 0.1         #Define decay
for i in range(len(train_data)):  
    Smoothing = gamma*train_data[i] + (1-gamma)*Smoothing   # Update smoothing Value  
    train_data[i] = Smoothing # Replace datapoint with smoothened value

In [ ]:
window_size = 26            # Define window size
N = train_data.size         # and length of observations

std_avg_predictions = []    # Empty list to catch std
mse_errors = []             # and mse

for i in range(window_size,N):
    # Append the standard mean per window
    std_avg_predictions.append(np.mean(train_data[i-window_size:i]))                                                                                                         
 
    # Compute mean squared error per batch 
    mse_errors.append((std_avg_predictions[-1]-train_data[i])**2) 

print('MSE error for standard averaging: %.5f' % (0.5*np.mean(mse_errors)))

In [ ]:
plt.figure(figsize = (19,6))
plt.plot(range(train_data.shape[0]),train_data,color='darkblue',label='Actual')
plt.plot(range(window_size,N),std_avg_predictions,color='orange',label='Predicted')
plt.xticks(range(0,df.shape[0]-len(test_data),50),df.index[::50],rotation=45)

plt.xlabel('Date')
plt.ylabel('Mid Price')
plt.legend(fontsize=18)
plt.show()

In [ ]:
ema_avg_predictions = []
mse_errors = []

EMA = 0.0
ema_avg_predictions.append(EMA)

gamma = 0.9
window_size = 100
N = len(train_data)

for i in range(1,N):
    EMA = EMA*gamma + (1.0-gamma)*train_data[i-1]
    ema_avg_predictions.append(EMA)
    mse_errors.append((ema_avg_predictions[-1]-train_data[i])**2)

print('MSE error for EMA averaging: %.5f'%(0.5*np.mean(mse_errors)))

In [ ]:
def create_dataset(dataset, look_back=7, foresight=3):   
    X, Y = [], []
    for i in range(len(dataset)-look_back-foresight): 
        obs = dataset[i:(i+look_back), 0] # Sequence of 7 stock prices as features forming an observation                                        
       # Append sequence
        X.append(obs)
       # Append stock price value occurring 4 time-steps into future
        Y.append(dataset[i + (look_back+foresight), 0]) 
    return np.array(X), np.array(Y)

In [ ]:
x_train, y_train = create_dataset(train_data)
x_test, y_test = create_dataset(test_data)
print(x_train.shape)
x_train

In [ ]:
x_train = np.reshape(x_train, (x_train.shape[0], 1,  x_train.shape[1]))  #(990,1,7)
x_test = np.reshape(x_test, (x_test.shape[0], 1,  x_test.shape[1]))

In [ ]:
print(x_train)

In [ ]:
from keras.models import Sequential
from keras.layers import LSTM, GRU, Dense
from keras.layers import Dropout, Flatten

from keras.callbacks import ModelCheckpoint, EarlyStopping

In [ ]:
def feed_forward():
    model = Sequential()
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(1, activation='linear'))
    model.compile(loss='mae', optimizer='adam')
    return model

def simple_gru():
    model = Sequential()
    model.add(GRU(128,  input_shape=(1, 7), dropout=0.1, recurrent_dropout=0.1))
    
    model.add(Dense(1, activation='linear'))
    model.compile(loss='mse', optimizer='adam', metrics = 
                  ['mse']) 
#   model.compile(loss='mae', optimizer='adam', metrics = 
#                   ['mean_absolute_error']) 
    return model

def CNN():
    model = Sequential()
    model.add(input_shape=(7,))
    model.add(Conv1D(100, 10, activation='relu', input_shape=(TIME_PERIODS, num_sensors)))
    model.add(Conv1D(100, 10, activation='relu'))
    model.add(MaxPooling1D(3))
    model.add(Conv1D(160, 10, activation='relu'))
    model.add(Conv1D(160, 10, activation='relu'))
    model.add(GlobalAveragePooling1D())
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))
    return model

def simple_lstm():
    model = Sequential()
    model.add(LSTM(256, input_shape=(1, 7)))
    
    model.add(Dense(1, activation='linear'))
    
    model.compile(loss='mae', optimizer='adam')
    return model

def lstm_stacked():
    model = Sequential()
    model.add(LSTM(16, input_shape=(1, 7), dropout=0.1, recurrent_dropout=0.2, return_sequences=True))
    model.add(LSTM(16, dropout=0.1, recurrent_dropout=0.2))
    
    model.add(Dense(1, activation='linear'))
    
    model.compile(loss='mae', optimizer='adam')
    return model

In [ ]:
def plot_losses(network):
    plt.plot(network.history['loss'], label='loss')
    plt.plot(network.history['val_loss'], label='val loss')
    plt.legend()
    plt.show()

In [ ]:
def plot_predictions(model, y_test=y_test):
    
    preds = model.predict(x_test)
    plt.figure(figsize = (12,6))
    plt.plot(scaler.inverse_transform(preds.reshape(-1,1)), 
             label='generated', color='orange')
    plt.plot(scaler.inverse_transform(y_test.reshape(-1,1)),   
             label='Actual')
    plt.legend()
    print(type(scaler.inverse_transform(preds.reshape(-1,1))))
    print(scaler.inverse_transform(preds.reshape(-1,1)))
#     p_price = pd.Series(scaler.inverse_transform(preds.reshape(-1,1).reshape(1,-1)))
#     p_price.shape()
# #     y_test.shape()
#     pv = pd.DataFrame({'Predict': p_price, 
#                         'Actual': y_test
#                        })
#     print(pv)

In [ ]:
def train_network(list, x_train, y_train, epochs=5):
    for net in list:               
        
        network_name = str(net).split(' ')[1]
#         filepath = network_name + "_epoch-{epoch:02d}-loss-{loss:.4f}-.hdf5"
        print('Training:', network_name)
#         checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=0, save_best_only=True, mode='min')
        callbacks_list = [TensorBoard(log_dir='./tmp/log')]
        model = net()                  
        
        network = model.fit(x_train, y_train,
                            validation_split=0.2,
                            epochs=epochs,
                            batch_size=64,
                            callbacks=callbacks_list)
        model.summary()
        score = model.evaluate(x_test, y_test, verbose=1)
        print("Test score", score[0])
        print("Test accuracy", score[1])
        plot_predictions(model, y_test)
        plot_losses(network)
        
    return network, model



In [ ]:
y_test.shape

In [ ]:
all_networks = [CNN]

# all_networks = [feed_forward, simple_gru, simple_lstm, lstm_stacked]
train_network(all_networks, x_train, y_train, epochs=3)